# Covid19 Analysis for Nordic Countries

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [24]:
df = pd.read_json('https://pomber.github.io/covid19/timeseries.json')[['Sweden','Norway','Denmark','Finland']]

In [25]:
n_rows = df.shape[0]
df['Sweden'][0]

{'date': '2020-1-22', 'confirmed': 0, 'deaths': 0, 'recovered': 0}

In [26]:
# extract dates
dates = []
for i in range(n_rows):
    dates.append(df['Sweden'][i]['date'])

In [27]:
df['Date'] = dates
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [28]:
df.tail()

,Sweden,Norway,Denmark,Finland
Date,,,,
2020-03-24,"{'date': '2020-3-24', 'confirmed': 2286, 'deat...","{'date': '2020-3-24', 'confirmed': 2863, 'deat...","{'date': '2020-3-24', 'confirmed': 1718, 'deat...","{'date': '2020-3-24', 'confirmed': 792, 'death..."
2020-03-25,"{'date': '2020-3-25', 'confirmed': 2526, 'deat...","{'date': '2020-3-25', 'confirmed': 3084, 'deat...","{'date': '2020-3-25', 'confirmed': 1862, 'deat...","{'date': '2020-3-25', 'confirmed': 880, 'death..."
2020-03-26,"{'date': '2020-3-26', 'confirmed': 2840, 'deat...","{'date': '2020-3-26', 'confirmed': 3369, 'deat...","{'date': '2020-3-26', 'confirmed': 2023, 'deat...","{'date': '2020-3-26', 'confirmed': 958, 'death..."
2020-03-27,"{'date': '2020-3-27', 'confirmed': 3069, 'deat...","{'date': '2020-3-27', 'confirmed': 3755, 'deat...","{'date': '2020-3-27', 'confirmed': 2200, 'deat...","{'date': '2020-3-27', 'confirmed': 1041, 'deat..."
2020-03-28,"{'date': '2020-3-28', 'confirmed': 3447, 'deat...","{'date': '2020-3-28', 'confirmed': 4015, 'deat...","{'date': '2020-3-28', 'confirmed': 2366, 'deat...","{'date': '2020-3-28', 'confirmed': 1167, 'deat..."


In [29]:
df_deaths = pd.DataFrame(index=df.index)
for col in df.columns:
    df_deaths[col] = [c.get('deaths') for c in df[col]]

In [30]:
# Start from March 10 before first deaths
df_deaths = df_deaths['2020-03-10':]

In [31]:
# population data from Wikipedia

df_pop = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_population_(United_Nations)')[3]
df_pop = df_pop[['Country or area', 'Population(1 July 2019)']]

df_pop['Country or area'] = df_pop['Country or area'].str.replace('\[.*\]','')
df_pop = df_pop.pivot_table(columns='Country or area',values='Population(1 July 2019)')[df.columns]
df_pop = df_pop / 1000000

In [32]:
df_pop['Sweden']

Population(1 July 2019)    10.036379
Name: Sweden, dtype: float64

In [33]:
df_deaths_per_mn = pd.DataFrame(index=df_deaths.index)
for col in df_deaths.columns:
    df_deaths_per_mn[col] = df_deaths[col] / df_pop[col].values

In [51]:
df_deaths_per_mn.tail()

,Sweden,Norway,Denmark,Finland
Date,,,,
2020-03-24,3.586951,2.230957,5.544125,0.180761
2020-03-25,6.177527,2.602783,5.890632,0.542284
2020-03-26,7.672090,2.602783,7.103410,0.903807
2020-03-27,10.461941,3.532349,9.009203,1.265329
2020-03-28,10.461941,4.276001,11.261503,1.626852


# Trend from first death

In [77]:
df_deaths_1 = df_deaths[df_deaths != 0]

# Remove all dates with zero deaths
df_deaths_1 = df_deaths_1.apply(lambda x: pd.Series(x.dropna().values))

# Add zero to first day
df_deaths_1 = pd.concat([pd.DataFrame(np.zeros((1,df_deaths_1.shape[1])),columns=df_deaths_1.columns), df_deaths_1], axis=0,ignore_index=True)

In [78]:
# deaths per mn inhabitants since first death
df_deaths_per_mn_1 = pd.DataFrame(index=df_deaths_1.index)
for col in df_deaths.columns:
    df_deaths_per_mn_1[col] = df_deaths_1[col] / df_pop[col].values

# Plots

## Deaths over time

In [79]:
import plotly.graph_objects as go
import pandas as pd
import plotly


fig = go.Figure()

for col in df_deaths:
    fig.add_trace(go.Scatter(x=df_deaths.index, y=df_deaths[col], name=col))

fig.update_layout(title_text='COVID19 Nordic Deaths, starting March 10 2020',hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths.html')

'deaths.html'

In [80]:
fig = go.Figure()

for col in df_deaths_per_mn:
    fig.add_trace(go.Scatter(x=df_deaths_per_mn.index, y=df_deaths_per_mn[col], name=col))

fig.update_layout(title_text='COVID19 Nordic Deaths per Mn inhabitants, starting March 10 2020',hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths_mn.html')

'deaths_mn.html'

In [81]:
fig = go.Figure()

for col in df_deaths_1:
    fig.add_trace(go.Scatter(x=df_deaths_1.index, y=df_deaths_1[col], name=col))

fig.update_layout(template='plotly_white', title_text='COVID19 Nordic Deaths, increase per day since first death',hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths_1.html')

'deaths_1.html'

In [82]:
fig = go.Figure()

for col in df_deaths_per_mn_1:
    fig.add_trace(go.Scatter(x=df_deaths_per_mn_1.index, y=df_deaths_per_mn_1[col], name=col))

fig.update_layout(template='plotly_white', title_text='COVID19 Nordic Deaths per Mn inhabitants, increase per day since first death', hovermode = 'x',
                  xaxis_rangeslider_visible=True)
fig.show()
plotly.offline.plot(fig, filename='deaths_mn_1.html')

'deaths_mn_1.html'